In [1]:
%matplotlib tk
from Tkinter import *
import tkFileDialog
from PIL import Image
from PIL import ImageStat
from PIL import ImageFilter
import numpy as np
import socket
import time
import StringIO

#open file dialog
def openImage(initialdir):
    root = Tk()
    root.withdraw()
    filename = tkFileDialog.askopenfilename(initialdir='demo_images/')
    return filename
#convert filepath into real string vectors can be send to feature receiver
def filepathToString(filepath):
    binSize = 16
    im = Image.open(filepath)
    size = im.size
    t = size[0]*size[1]
    #size0 is width,y.size1 is height x
    im = im.filter(ImageFilter.SMOOTH)
    #convert to RGB channels
    l1,l2,l3 = im.split()
    l1 = np.array(l1.getdata())
    l2 = np.array(l2.getdata())
    l3 = np.array(l3.getdata())
    #the order is R/G/B
    pixels = zip(l1,l2,l3)
    digit = 256/binSize
    histogram = np.zeros(digit**3)
    for pixel in pixels:
        hundredth = int(pixel[2]/binSize)
        tenth = int(pixel[1]/binSize)
        numth = int(pixel[0]/binSize)
        order = hundredth*(digit**2)+tenth*digit+numth
        histogram[order] += 1
    s = ''
    for i,j in enumerate(histogram):
        if j!=0:
            s += (str(i) + ':' + str(j/float(np.prod(size))) +',')
    return s

#test for send image data to feature receiver
def serverSender(host='localhost',port=9999):
    while True:
        try:
            serverSocket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
            serverSocket.bind((host,port))
            serverSocket.listen(1)
            try:
                while True:
                    connection,address = serverSocket.accept()
                    connection.settimeout(10000)
                    while True:
                        path = openImage('images/')
                        if path!='':
                            data = filepathToString(path)
                            try:
                                #send the original image name
                                imageName = path.split('/')[-1]
                                print connection.send( imageName + '\t' + data +'\n')
                                buf = connection.recv(128)
                                if buf!='':
                                    #here we need to take over wrong input, right now just ignore this
                                    buf = StringIO.StringIO(buf)
                                    buf.read(2)
                                    buf = buf.read()
                                    print "get " + buf
                                else:
                                    break
                                time.sleep(2)
                            except Exception,e:
                                print 'exception 1'
                                print str(e)
                        else:
                            print "waiting for next query"
                            time.sleep(2)
            except socket.timeout:
                print 'timeout'
            except socket.error,e:
                print e
                print "socket ends from other side"
                #right now, not handling errors
                pass
            except KeyboardInterrupt:
                print "catch keyinterp???"
            finally:
                print "finnally 1"
                connection.close()
                serverSocket.close()
                break
        except KeyboardInterrupt:
            print "catch keyinterp???"
            serverSocket.close()
            break
        finally:
            print "finally 2"
            serverSocket.close()

In [2]:
serverSender()

7597
get 0121.tif received
6445
get 1437.jpg received
catch keyinterp???
finnally 1
finally 2
